# Pracownia z analizy numerycznej, zadanie P1.16

## Metoda Newtona

In [1]:
IJulia.load("newton.jl")

In [6]:
include("utils.jl")
include("gauss.jl")

function newton(F, J, X
                ; solve = gauss!
                , ϵs = 1e-15
                , maxiter = 20
                , log = false
                , latex = false)
  ϵ = 1.0
  i = 0
  if log
    @printf("Iteration\t||F(X)||\t\tϵ\n")
  elseif latex
    @printf "Iteracja & \$\\|F(X)\\|\$ & \$\\epsilon\$ \\\\ \\hline\n"
  end
  while ϵ > ϵs && i < maxiter
    JX = J(X)
    FX = F(X)
    δX = solve(JX, FX)
    ϵ  = norm(δX) / norm(X)
    if log
      @printf("%d\t\t%e\t%e\n", i, norm(FX), ϵ)
    elseif latex
      @printf "%d & %e & %e \\\\ \\hline\n" i norm(FX) ϵ
    end
    X  = X - δX
    i += 1
  end
  if log
    @printf("\nFinal approximation:\n")
    print_array(X)
  end
end


newton (generic function with 1 method)

In [9]:
gauss!([1.0 2.0; 3.0 4.0], [3.0, 4.0])

2-element Array{Float64,1}:
 -2.0
  2.5

In [47]:
include("examples.jl")

Examples

### Przykład 1
\begin{align*}
	f_1(x) &= (x_1)^2 + (x_2)^2 - 25 \\
	f_2(x) &= (x_1)^2 - x_2 - 2
\end{align*}

In [13]:
F, J, X = Examples.set_1()
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		3.114482e+01	5.884301e+00
1		5.301945e+01	3.675183e-01
2		9.411666e+00	1.725992e-01
3		7.950828e-01	2.592261e-02
4		1.475216e-02	5.889474e-04
5		8.572156e-06	3.413509e-07
6		2.913225e-12	1.158651e-13
7		1.256074e-15	3.663005e-17

Final approximation:
[ 2.514e+00 4.322e+00 ]


### Przykład 2
\begin{align*}
	f_1(x) &= x_1 + x_2 + x_3 \\
	f_2(x) &= (x_1)^2 + (x_2)^2 + (x_3)^2 - 2 \\
    f_3(x) &= x_1(x_2 + x_3) + 1
\end{align*}

In [14]:
F, J, X = Examples.set_2()
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		1.873726e+00	1.076430e+00
1		8.631775e-01	2.001541e-01
2		9.520625e-02	3.428260e-02
3		1.864611e-03	7.209369e-04
4		7.757667e-07	3.004528e-07
5		1.344542e-13	5.211241e-14
6		0.000000e+00	0.000000e+00

Final approximation:
[ 1.000e+00 -2.812e-17 -1.000e+00 ]


### Przykład 3
\begin{align*}
	f_1(x) &= x_1 x_2 - (x_3)^2 - 1 \\
	f_2(x) &= x_1 x_2 x_3 + (x_2)^2 - (x_1)^2 - 2 \\
    f_3(x) &= e^{x_1} + x_3 - e^{x_2} - 3
\end{align*}

In [15]:
F, J, X = Examples.set_3()
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		1.393168e+00	5.927076e-01
1		5.404950e-01	7.379226e-02
2		2.799916e-02	4.400197e-03
3		1.086581e-04	1.534432e-05
4		1.806242e-09	2.150888e-10
5		1.158023e-15	7.605550e-17

Final approximation:
[ 1.681e+00 1.420e+00 1.178e+00 ]


### Przykład 4
\begin{align*}
	f_1(x) &= (x_1)^2 \\
	f_2(x) &= (x_2)^2 
\end{align*}

In [17]:
F, J, X = Examples.set_4()
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		1.414214e+00	5.000000e-01
1		3.535534e-01	5.000000e-01
2		8.838835e-02	5.000000e-01
3		2.209709e-02	5.000000e-01
4		5.524272e-03	5.000000e-01
5		1.381068e-03	5.000000e-01
6		3.452670e-04	5.000000e-01
7		8.631675e-05	5.000000e-01
8		2.157919e-05	5.000000e-01
9		5.394797e-06	5.000000e-01
10		1.348699e-06	5.000000e-01
11		3.371748e-07	5.000000e-01
12		8.429370e-08	5.000000e-01
13		2.107342e-08	5.000000e-01
14		5.268356e-09	5.000000e-01
15		1.317089e-09	5.000000e-01
16		3.292723e-10	5.000000e-01
17		8.231806e-11	5.000000e-01
18		2.057952e-11	5.000000e-01
19		5.144879e-12	5.000000e-01

Final approximation:
[ 9.537e-07 9.537e-07 ]


### Przykład 5
\begin{align*}
	f_1(x) &= (x_1)^3 + 7(x_2)^2 - 7 \\
	f_2(x) &= (x_1)^2 + (x_2)^2 - 1
\end{align*}

In [18]:
F, J, X = Examples.set_5()
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		4.472136e+00	1.264911e+00
1		2.031359e+01	8.820599e-01
2		1.371753e+01	4.438798e-01
3		2.183828e+00	3.083473e-01
4		2.451219e-01	2.325736e-01
5		8.822255e-02	1.094101e-01
6		1.366937e-02	5.328404e-02
7		2.924320e-03	2.613764e-02
8		6.879729e-04	1.293397e-02
9		1.675726e-04	6.432501e-03
10		4.139445e-05	3.207550e-03
11		1.028945e-05	1.601591e-03
12		2.565159e-06	8.002482e-04
13		6.404014e-07	3.999872e-04
14		1.599900e-07	1.999594e-04
15		3.998376e-08	9.997112e-05
16		9.994225e-09	4.998342e-05
17		2.498342e-09	2.499117e-05
18		6.245589e-10	1.249545e-05
19		1.561364e-10	6.247696e-06

Final approximation:
[ 6.248e-06 1.000e+00 ]


In [20]:
F, J, X = Examples.generalized_rosenbrock(4)
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		1.414214e+00	Inf
1		1.414214e+01	1.000000e+00
2		0.000000e+00	0.000000e+00

Final approximation:
[ 1.000e+00 1.000e+00 1.000e+00 1.000e+00 ]


### Przykład 6
Układ równań postaci
$$ f_k(x) = x_1 + (k+1)x_2 + \ldots + (k+1)^{n-1} - \frac{(k+1)^n - 1}{k} $$
których Jakobian tworzy macierze podobne do macierzy Vandermonde'a, rowiązaniem jest $ x = (1 \ldots 1)^T $

In [43]:
n = 10
F, J = Examples.vandermonde(n)
X = rand(n)
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		1.109677e+09	1.044748e+00
1		5.835779e-08	3.591647e-06
2		1.496403e-07	3.966258e-05
3		1.561474e-07	1.324132e-05
4		4.850791e-07	4.215657e-06
5		5.626922e-07	4.916246e-06
6		5.882738e-07	2.774697e-05
7		5.626922e-07	4.916248e-06
8		8.031784e-08	7.014734e-07
9		0.000000e+00	0.000000e+00

Final approximation:
[ 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 ]


In [45]:
n = 15
F, J = Examples.vandermonde(n)
X = rand(n)
newton(F, J, X, log = true)

Iteration	||F(X)||		ϵ
0		3.929309e+16	8.395358e+03
1		7.957300e+00	2.425639e-02
2		5.000000e-01	6.571320e-01
3		3.871994e+00	4.621210e-01
4		4.003055e+00	1.982333e-01
5		1.147562e+01	2.773830e-01
6		8.723854e+00	2.223422e-01
7		5.000000e-01	3.940746e-01
8		7.818537e-01	5.209915e-01
9		0.000000e+00	0.000000e+00

Final approximation:
[ 2.471e+03 -5.695e+03 5.855e+03 -3.564e+03 1.443e+03 -4.099e+02 8.623e+01 -1.210e+01 2.502e+00 8.720e-01 1.008e+00 9.996e-01 1.000e+00 1.000e+00 1.000e+00 ]


Zauważmy, że dla większych n wynik algorytmu jest diametralnie różny od prawdziwego rozwiązania

In [86]:
setprecision(256)
F, J, X = Examples.polynomial(3, BigFloat)
newton(F, J, [BigFloat(1), BigFloat(1), BigFloat(1)], log = true, ϵs = big"1e-50")

Iteration	||F(X)||		ϵ
0		1.728153e+03	5.649324e+01
1		6.684426e+04	4.840644e-01
2		1.624106e+04	4.405802e-01
3		3.638117e+03	3.139514e-01
4		6.152705e+02	1.147942e-01
5		4.525508e+01	1.165982e-02
6		4.081371e-01	1.161528e-04
7		3.809541e-05	1.098992e-08
8		3.362493e-13	9.706921e-17
9		2.621116e-29	7.566231e-33
10		1.592618e-61	4.597436e-65

Final approximation:
[ 8.892e+00 8.509e+00 1.284e+01 ]


In [21]:
IJulia.load("gauss.jl")

In [ ]:
"""
Oblicza rozwiązanie układu równań AX = B metodą eliminacji Gaussa.

Korzysta z naiwnego algorytmu zaadaptowanego z książki Numerical Mathematics
and Computing Warda Cheney'a i Davida Kincaida
"""
function gauss_naive!(A, B)
  n = length(B)
  X = Array{eltype(A)}(n)
  for k in 1 : n-1
    for i = k + 1 : n
      mult = A[i, k] / A[k, k]
      A[i, k] = 0 
      for j in k + 1 : n
        A[i, j] -= mult * A[k, j]
      end
      B[i] -= mult * B[k]
    end
  end
  X[n] = B[n] / A[n, n]
  for i in n-1 : -1 : 1
    sum = B[i]
    for j in i + 1 : n
      sum -= A[i, j] * X[j]
    end
    X[i] = sum / A[i, i]
  end
  X
end


"""
Oblicza rozwiązanie układu równań AX = B metodą eliminacji Gaussa.

Korzysta z algorytmu skalowanego elementu głównego, przekształca
macierz wejściową A, tak że w górnej trójkątnej jej części znajduje się
jej postać schodkowa, natomiast w dolnej zapisane są użyte mnożniki
które wykorzystywane są w kroku podstawiania

Algorytm został zaadaptowany z książki Numerical Mathematics and Computing
Warda Cheney'a i Davida Kincaida
"""
function gauss!(A, B, log = false)
  n = length(B)
  l = gauss_eliminate!(A, n)
  if log
    @show A
    @show l
  end
  X = gauss_substitute!(A, B, l, n)
  if log
    @show X
  end
  X
end


"""
Implementuje krok eliminacji, zapisując mnożniki w dolnej trójkątnej
części macierzy A i zwracając permutację wierszy macierzy A
"""
function gauss_eliminate!(A, n)
  s = Array{eltype(A)}(n) # Scale vector
  l = Array{Int}(n)       # Index permutation
  for i in 1 : n
    l[i] = i
    smax = 0
    for j in 1 : n
      smax = max(smax, abs(A[i, j]))
    end
    s[i] = smax
  end
  for k in 1 : n-1
    rmax = 0
    maxidx = k
    for i in k : n
      r = abs(A[l[i], k] / s[l[i]])
      if r > rmax
        rmax = r
        maxidx = i
      end
    end
    t = l[maxidx]
    l[maxidx] = l[k]
    l[k] = t
    for i = k + 1 : n
      mult = A[l[i], k] / A[l[k], k]
      A[l[i], k] = mult 
      for j in k + 1 : n
        A[l[i], j] -= mult * A[l[k], j]
      end
    end
  end
  l
end

"""
Implementuje krok podstawiania
"""
function gauss_substitute!(A, B, l, n)
  X = Array{eltype(A)}(n) # Solution vector
  for k in 1 : n - 1
    for i in k + 1 : n
      B[l[i]] -= A[l[i], k] * B[l[k]]
    end
  end
  X[n] = B[l[n]] / A[l[n], n]
  for i in n-1 : -1 : 1
    sum = B[l[i]]
    for j in i + 1 : n
      sum -= A[l[i], j] * X[j]
    end
    X[i] = sum / A[l[i], i]
  end
  X
end
